## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [12]:
import pandas as pd

In [13]:
df=pd.read_csv('fake-news/train.csv')

In [16]:
print(df.shape)

(20800, 5)


In [17]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [18]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [19]:
###Drop Nan Values
df=df.dropna()


In [20]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [21]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [22]:
## Get the Dependent features
y=df['label']

In [23]:
X.shape

(18285, 4)

In [24]:
y.shape

(18285,)

In [30]:
import tensorflow as tf
tf.__version__
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [31]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [32]:
messages=X.copy()

In [39]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [53]:
messages.reset_index(inplace=True)
messages

,level_0,index,id,title,author,text
0,0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...,...
18280,18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [54]:
import nltk
import re
from nltk.corpus import stopwords

In [55]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [57]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  #print(i)
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i]) #except a to z and A to Z, all other special characters are removed
  review = review.lower() # to maintain same case
  review = review.split() # words

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')] # list of words, if a word isnt present in stopwords -> ignore the word
  review = ' '.join(review)
  corpus.append(review)

In [58]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [59]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [60]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[314, 515, 3038, 4473, 2076, 3841, 1359, 3518, 2543, 2890],
 [4081, 1633, 1645, 2286, 4449, 486, 4730],
 [1811, 3995, 3796, 4225],
 [2953, 2590, 3031, 827, 283, 2466],
 [286, 4449, 677, 1083, 1557, 3449, 4449, 370, 1088, 2161],
 [2670,
  1614,
  1044,
  301,
  1282,
  116,
  2144,
  449,
  4485,
  2422,
  531,
  4474,
  4573,
  2950,
  4730],
 [1157, 3633, 4255, 2649, 4941, 2193, 4434, 4814, 1923, 3976, 1983],
 [3977, 3013, 2421, 4602, 884, 4632, 116, 4114, 1923, 3976, 1983],
 [3368, 3504, 4988, 2190, 4308, 410, 4266, 1297, 116, 2526],
 [3850, 4644, 1110, 4054, 831, 4541, 2839, 1934],
 [1686, 1031, 4848, 1158, 4707, 177, 2944, 2179, 2931, 4693, 3564],
 [827, 4937, 2076, 410, 116, 884],
 [3303, 1610, 2293, 3842, 2673, 1163, 4951, 4232, 2905],
 [3185, 210, 4886, 2153, 3543, 1045, 3436, 1923, 3976, 1983],
 [445, 2623, 4773, 1185, 387, 1923, 3976, 1983],
 [3742, 565, 1035, 2177, 2959, 1414, 280, 2027, 2651, 4489],
 [1283, 3409, 1633],
 [181, 1962, 664, 2561, 116, 1625, 1657, 4730],
 [4843

In [61]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [62]:
onehot_repr[1]

[4081, 1633, 1645, 2286, 4449, 486, 4730]

### Embedding Representation

In [63]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[ 314  515 3038 ...    0    0    0]
 [4081 1633 1645 ...    0    0    0]
 [1811 3995 3796 ...    0    0    0]
 ...
 [2603 2471 3026 ...    0    0    0]
 [ 356  884 3611 ...    0    0    0]
 [2179 4078 2816 ...    0    0    0]]


In [64]:
embedded_docs[1]

array([4081, 1633, 1645, 2286, 4449,  486, 4730,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [65]:
embedded_docs[0]

array([ 314,  515, 3038, 4473, 2076, 3841, 1359, 3518, 2543, 2890,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [66]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [67]:
len(embedded_docs),y.shape

(18285, (18285,))

In [68]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [69]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [70]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [71]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 16s 64ms/step - loss: 0.3188 - accuracy: 0.8430 - val_loss: 0.2012 - val_accuracy: 0.9143
Epoch 2/10
192/192 [==============================] - 4s 18ms/step - loss: 0.1469 - accuracy: 0.9423 - val_loss: 0.2045 - val_accuracy: 0.9176
Epoch 3/10
192/192 [==============================] - 3s 14ms/step - loss: 0.1028 - accuracy: 0.9620 - val_loss: 0.2618 - val_accuracy: 0.9125
Epoch 4/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0728 - accuracy: 0.9742 - val_loss: 0.2411 - val_accuracy: 0.9087
Epoch 5/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0496 - accuracy: 0.9834 - val_loss: 0.2979 - val_accuracy: 0.9084
Epoch 6/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0397 - accuracy: 0.9882 - val_loss: 0.3393 - val_accuracy: 0.9125
Epoch 7/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0264 - accuracy: 0.9915 - val_loss: 0.3729 - val_accuracy: 0.9059

### Adding Dropout

In [72]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [73]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 2ms/step


In [74]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [75]:
from sklearn.metrics import confusion_matrix

In [76]:
confusion_matrix(y_test,y_pred)

array([[3419,    0],
       [2616,    0]])

In [77]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5665285832642917

In [78]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.57      1.00      0.72      3419
           1       0.00      0.00      0.00      2616

    accuracy                           0.57      6035
   macro avg       0.28      0.50      0.36      6035
weighted avg       0.32      0.57      0.41      6035



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
